In [ ]:
# from base import *
import sys
from PIL import Image
import json
import pickle
import socket
import cv2
import numpy as np
import time
import os
import datetime
from keras.models import load_model

from warp import transform_image, fvec2pose, featureize_input, get_min_pose, get_quest_fvec
from Stitcher import Stitcher
from SocketToUnity import OutputBroadcaster
from WebcamVideoStream import WebcamVideoStream

import tempfargs as fargs

dir_path = os.path.dirname(os.path.realpath(__file__))
sys.path.append("/home/bpe/VR_Controller_BPE/openpose/build/python")
from openpose import pyopenpose as op

if __name__ == '__main__':

    stitcher = Stitcher()
    broadcaster = OutputBroadcaster()
    model = load_model('/home/bpe/VR_Controller_BPE/src/core/lift_2d_to_3d_whole_body.h5')
    h_left = np.array([0.1625,-1.05,0])
    h_right = np.array([-0.1625,-1.05,0])

    video_fps = 0

    if fargs.input_vid_no_stich:
        single_vid = WebcamVideoStream(src=fargs.input_vid_no_stich_path, vid=True)
        video_fps = single_vid.cap()
    else:
        if fargs.input_from_video:
            rtop = WebcamVideoStream(src=fargs.rtop, vid=True)
            rbot = WebcamVideoStream(src=fargs.rbot, vid=True)
            ltop = WebcamVideoStream(src=fargs.ltop, vid=True)
            lbot = WebcamVideoStream(src=fargs.lbot, vid=True)
            video_fps = rtop.cap()
        if fargs.input_live:
            rtop = WebcamVideoStream(6).start()
            rbot = WebcamVideoStream(0).start()
            ltop = WebcamVideoStream(4).start()
            lbot = WebcamVideoStream(2).start()

    print('Initialization is done')

    fshape = (1420, 730)
    wshape = (1640, 800)
    if fargs.outputFilePath is not None:
        if fargs.input_vid_no_stich:
            head, tail = os.path.split(fargs.input_vid_no_stich_path)
            folder_name = tail.split(".")[0]
        else:
            folder_name = fargs.output_folder
        log_path = fargs.outputFilePath + "/" + folder_name + "/"
        video_path = fargs.outputFilePath + "/" + folder_name + "/" + folder_name + "_output_video"
        video_in_path = fargs.outputFilePath + "/" + folder_name + "/" + folder_name + "_input_video"
        data_log_path = fargs.outputFilePath + "/" + folder_name + "/data_log/"
        os.makedirs(log_path, exist_ok=True)
        if fargs.saveOutputDataLog:
            os.makedirs(data_log_path, exist_ok=True)
        if fargs.saveOutputVideo:
            outvid = cv2.VideoWriter(video_path + ".avi", cv2.VideoWriter_fourcc(*'XVID'), 10.0, fshape)
            outvid_perspective = cv2.VideoWriter(dname + "_vid_perspective.avi",  cv2.VideoWriter_fourcc(*'XVID'), 10.0, fshape)
        if fargs.saveInputVideo4cam:
            out_inFrame = cv2.VideoWriter(video_in_path + ".avi", cv2.VideoWriter_fourcc(*'XVID'), 10.0, (1280,960))
            perspective_in = cv2.VideoWriter(dname + "_vid_perspective_input.avi",  cv2.VideoWriter_fourcc(*'XVID'), 10.0, wshape)

    framecount = 0
    params = dict()
    params["model_folder"] = "/home/bpe/VR_Controller_BPE/openpose/models/"
    opWrapper = op.WrapperPython()
    opWrapper.configure(params)
    opWrapper.start()
    while True:
        start = time.time()

        print("Received data:", broadcaster.dataStorage[0])

        if fargs.input_vid_no_stich:
            ret, frame =  single_vid.read()
            if not (ret):
                print("Wrapping up")
                if fargs.outputFilePath is not None:
                    if fargs.saveOutputVideo:
                        outvid.release()
                        outvid_perspective.release()
                break

            w = int(frame.shape[1]/2)
            h = int(frame.shape[0]/2)
            rtopf = frame[0:h,0:w]
            ltopf = frame[0:h,w:2*w]
            rbotf = frame[h:2*h,0:w]
            lbotf = frame[h:2*h,w:2*w]
        else:
            # polling from all 4 cameras
            ret1, rtopf = rtop.read()
            ret2, rbotf = rbot.read()
            ret3, ltopf = ltop.read()
            ret4, lbotf = lbot.read()
            if fargs.input_live:
                rtopf = cv2.flip(rtopf, 0)
                rbotf = cv2.flip(rbotf, 0)
                ltopf = cv2.flip(ltopf, 0)
                lbotf = cv2.flip(lbotf, 0)

            if not (ret1 or ret2 or ret3 or ret4):
                print("Wrapping up")
                rtop.stop()
                rbot.stop()
                ltop.stop()
                lbot.stop()
                if fargs.outputFilePath is not None:
                    if fargs.saveOutputVideo:
                        outvid.release()
                        outvid_perspective.release()
                    if fargs.saveInputVideo4cam:
                        out_inFrame.release()
                        perspective_in.release()
                break
            if not (ret1 and ret2):
                print("Right not grabbed")
                rtopf = np.zeros((rtopf.shape[0], rtopf.shape[1], 3), np.uint8)
                rbotf = np.zeros((rbotf.shape[0], rbotf.shape[1], 3), np.uint8)
            if not (ret3 and ret4):
                print("Left not grabbed")
                ltopf = np.zeros((ltopf.shape[0], ltopf.shape[1], 3), np.uint8)
                lbotf = np.zeros((lbotf.shape[0], lbotf.shape[1], 3), np.uint8)

            in_frame = np.vstack((np.hstack((rtopf,ltopf)),np.hstack((rbotf,lbotf))))
            if fargs.saveInputVideo4cam:
                out_inFrame.write(in_frame)

        framecount = framecount + 1
        # if framecount < 1000: continue;

        lframe, rframe = stitcher.stitch(rtopf, rbotf, ltopf, lbotf)
        pad_image = np.zeros((lframe.shape[0],20,3), np.uint8)
        lframe = np.hstack((lframe,pad_image))
        rframe = np.hstack((pad_image,rframe))
        frame = np.hstack((lframe,rframe))

        # OPENPOSE ON ORIGINAL STITCHING
        datum = op.Datum()
        datum.cvInputData = frame
        opWrapper.emplaceAndPop(op.VectorDatum([datum]))
        left_keypoints, right_keypoints = np.zeros((25,3)), np.zeros((25,3))
        orig_skeletal_output = np.zeros((fshape[1], fshape[0], 3))
        if datum.poseKeypoints is not None:
            keypoints = np.array(datum.poseKeypoints)
            scores = np.array(datum.poseScores)
            idx1 = np.argmax(scores)
            body1 = keypoints[idx1]
            scores[idx1] = 0
            idx2 = np.argmax(scores)
            body2 = keypoints[idx2]
            orig_skeletal_output = datum.cvOutputData
            if body1[0,0] < body2[0,0]:
                left_keypoints = body1
                right_keypoints = body2
            else:
                left_keypoints = body2
                right_keypoints = body1

        right_keypoints = right_keypoints - [lframe.shape[1],0,0]
        pose_orig = get_min_pose(left_keypoints,right_keypoints)

        # USING PERSPECTIVE WARP
        warp_lframe = transform_image(lframe,left_keypoints)
        warp_rframe = transform_image(rframe,right_keypoints)
        pad_image = np.zeros((warp_lframe.shape[0],20,3), np.uint8)
        lframe = np.hstack((warp_lframe,pad_image))
        rframe = np.hstack((pad_image,warp_rframe))
        frame = np.hstack((lframe,rframe))
        if fargs.saveInputVideo4cam:
            perspective_in.write(frame)

        datum = op.Datum()
        datum.cvInputData = frame
        opWrapper.emplaceAndPop(op.VectorDatum([datum]))
        left_keypoints, right_keypoints = np.zeros((25,3)), np.zeros((25,3))
        skeletal_output = np.zeros((wshape[1], wshape[0], 3))
        if datum.poseKeypoints is not None:
            keypoints = np.array(datum.poseKeypoints)
            scores = np.array(datum.poseScores)
            idx1 = np.argmax(scores)
            body1 = keypoints[idx1]
            scores[idx1] = 0
            idx2 = np.argmax(scores)
            body2 = keypoints[idx2]
            skeletal_output = datum.cvOutputData
            if body1[0,0] < body2[0,0]:
                left_keypoints = body1
                right_keypoints = body2
            else:
                left_keypoints = body2
                right_keypoints = body1
        right_keypoints = right_keypoints - [lframe.shape[1],0,0]
        pose_persepective = get_min_pose(left_keypoints,right_keypoints)

        # PROCESSING WITH CUSTOM MODEL
        best_pose_pts = get_min_pose(pose_persepective,pose_orig)
        fvec = featureize_input(np.array([best_pose_pts]))

        if broadcaster.dataStorage[0] == "":
            quest_vec = np.zeros((1,15))
        else:
            quest_vec = np.array([get_quest_fvec(broadcaster.dataStorage[0])])

        pred = model.predict([fvec,quest_vec])

        cols_to_remove = [0, 10, 12, 22, 25]
        cols_to_remove_vals = [1.0, 0.0, -1.0, 0.0, 1.0]
        pred = np.insert(pred, tuple([e-i for i, e in enumerate(cols_to_remove)]), 0., axis=1)
        for k in range(len(cols_to_remove)):
            pred[:,cols_to_remove[k]] = cols_to_remove_vals[k]

        pose = fvec2pose(pred)[0]
        print(pose.shape)

        print("FPS: ", 1/(time.time()-start))

        output_dict = {
            "input_live": fargs.input_live,
            "prerecorded_video_fps": video_fps,
            "frame_counter": framecount,
            "pose_timestamp": datetime.datetime.now().timestamp(),
            "combined_final_pose_smooth": pose
        }
        if fargs.outputFilePath is not None:
            if fargs.saveOutputDataLog:
                pickle_file_out = data_log_path + str(framecount) + ".pkl"
                with open(pickle_file_out, 'wb') as handle:
                    pickle.dump(output_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
            if fargs.saveOutputVideo:
                outvid.write(orig_skeletal_output)
                outvid_perspective.write(skeletal_output)
        if fargs.sendDatUnity:
            broadcaster.broadcastData(output_dict)
        if fargs.liveVisualize:
            # cv2.imshow("Skeletons", cv2.resize(orig_skeletal_output, None, fx=0.8, fy=0.8))
            cv2.imshow("Skeletons with warp", cv2.resize(skeletal_output, None, fx=0.6, fy=0.6))
            key = cv2.waitKey(1)
            if key == ord('q'):
                print("Wrapping up")
                if fargs.input_vid_no_stich:
                    single_vid.stop()
                else:
                    rtop.stop()
                    rbot.stop()
                    ltop.stop()
                    lbot.stop()
                if fargs.outputFilePath is not None:
                    if fargs.saveOutputVideo:
                        outvid.release()
                        outvid_perspective.release()
                    if fargs.saveInputVideo4cam:
                        out_inFrame.release()
                        perspective_in.release()
                break
